## Домашняя работа №3

#### Алгоритм Нидлмана-Вунша

In [2]:
seq1 = 'TCAAGCA'
seq2 = 'GCAAGA'
seq3 = 'GCATGACTTGAACCAGCTTACGATGACTAA'
seq4 = 'CAGACGACTTAGGCCCCAGCTAACGACGCCAA'

In [3]:
def needleman(seq1, seq2, match=1, gap=-2, mismatch=-1):
    n = len(seq1)
    m = len(seq2)
    dp = [[0] * (m + 1) for i in range(n + 1)]

    for i in range(n + 1):
        dp[i][0] = i * gap
    for j in range(m + 1):
        dp[0][j] = j * gap

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if seq1[i - 1] == seq2[j - 1]:
                diag = dp[i - 1][j - 1] + match
            else:
                diag = dp[i - 1][j - 1] + mismatch
            delete = dp[i - 1][j] + gap
            insert = dp[i][j - 1] + gap

            dp[i][j] = max(diag, delete, insert)

    res1, res2 = "", ""  #получим выровненные последовательности
    i, j = n, m
    while i > 0 or j > 0:     
        if i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch):
            res1 = seq1[i - 1] + res1
            res2 = seq2[j - 1] + res2
            i -= 1
            j -= 1
        elif i > 0 and dp[i][j] == dp[i - 1][j] + gap :
            res1 = seq1[i - 1] + res1
            res2 = "-" + res2
            i -= 1
        else:
            res1 = "-" + res1
            res2 = seq2[j - 1] + res2
            j -= 1

    return res1, res2, dp[n][m]

In [4]:
print(needleman(seq1,seq2))

('TCAAGCA', 'GCAAG-A', 2)


In [5]:
print(needleman(seq3,seq4))

('-GCATGACTT--GAACCAGCTTACGATGACTAA', 'CAGACGACTTAGGCCCCAGCTAACGACG-CCAA', 5)


#### Алгоритм Смита-Ватермана

In [6]:
def smith_waterman(seq1, seq2, match=1, gap=-2, mismatch=-1):
    n = len(seq1)
    m = len(seq2)
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    max_score = 0
    max_i, max_j = 0, 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if seq1[i - 1] == seq2[j - 1]:
                diag = dp[i - 1][j - 1] + match
            else:
                diag = dp[i - 1][j - 1] + mismatch

            delete = dp[i - 1][j] + gap
            insert = dp[i][j - 1] + gap

            dp[i][j] = max(0, diag, delete, insert)
 
            if dp[i][j] > max_score:
                max_score = dp[i][j]
                max_i, max_j = i, j

    res1, res2 = "", "" #получим выровненные последовательности
    i, j = max_i, max_j
    while dp[i][j] != 0:     
        if i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch):
            res1 = seq1[i - 1] + res1
            res2 = seq2[j - 1] + res2
            i -= 1
            j -= 1
        elif i > 0 and dp[i][j] == dp[i - 1][j] + gap :
            res1 = seq1[i - 1] + res1
            res2 = "-" + res2
            i -= 1
        else:
            res1 = "-" + res1
            res2 = seq2[j - 1] + res2
            j -= 1
    return res1, res2, dp[n][m]

In [7]:
print(smith_waterman(seq1,seq2))

('CAAG', 'CAAG', 3)


In [8]:
print(smith_waterman(seq3,seq4))

('CCAGCTTACGA', 'CCAGCTAACGA', 9)


Поскольку Алгоритм Смита-Ватермана ищет именно локальное выравнивание, нкулеотиды которые ухудшают результат выкидываются 

#### Проверим себя с готовыми реализациями глобальных выравниваний

In [30]:
from Bio import Align

aligner = Align.PairwiseAligner()
aligner.mode = 'global'  # 
aligner.match_score = 1  
aligner.mismatch_score = -1 
aligner.open_gap_score = -2  
aligner.extend_gap_score = -2

alignments = aligner.align(seq1, seq2)
print(alignments[0])  #выведем только первый вариант выравнивания
print(f"Score: {alignments[0].score}")

target            0 TCAAGCA 7
                  0 .||||-| 7
query             0 GCAAG-A 6

Score: 2.0


In [31]:
alignments = aligner.align(seq3, seq4)
print(alignments[0]) #выведем только первый вариант выравнивания
print(f"Score: {alignments[0].score}")

target            0 GC-ATGACTT-GAACC-AGCTTACGATGACTAA 30
                  0 ..-|.|||||-|..||-||||.||||.|.|-|| 33
query             0 CAGACGACTTAGGCCCCAGCTAACGACGCC-AA 32

Score: 5.0


Выводим только первые варианты, так как score везде одинаковый. Как мы видим алгоритмы написанные вручную дают тот же результат, что и реализованные пакеты Python